<a href="https://colab.research.google.com/github/adyanshkakran/Mechanistic_Interpretability/blob/main/InterpretingTransformers_String_concatenation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grokking Demo Notebook

<b style="color: red">To use this notebook, go to Runtime > Change Runtime Type and select GPU as the hardware accelerator.</b>

# Setup
(No need to read)

In [1]:
TRAIN_MODEL = True

In [2]:
!nvidia-smi

Fri Mar  1 17:18:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = True
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install transformer-lens
    %pip install circuitsvis

    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 11.2 MB/s eta 0:

In [4]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [5]:
pio.templates['plotly'].layout.xaxis.title.font.size = 20
pio.templates['plotly'].layout.yaxis.title.font.size = 20
pio.templates['plotly'].layout.title.font.size = 30

In [6]:
# Import stuff
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import os
import tqdm.auto as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from typing import List, Union, Optional
from functools import partial
import copy

# import itertools
# from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
# import dataclasses
# import datasets
# from IPython.display import HTML

In [7]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Plotting helper functions:

In [8]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [9]:
# Define the location to save the model, using a relative path
PTH_LOCATION = "workspace/_scratch/grokking_demo.pth"

# Create the directory if it does not exist
os.makedirs(Path(PTH_LOCATION).parent, exist_ok=True)

# Model Training

## Config

In [10]:
p = 36
frac_train = 0.7

# Optimizer config
lr = 1e-3
wd = 1.
betas = (0.9, 0.98)

num_epochs = 200
checkpoint_every = 20

DATA_SEED = 598

## Define Task
* Define modular addition
* Define the dataset & labels

In [17]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [18]:
import csv

In [19]:
# create string tensors to store the strings and labels
a_strings = []
b_strings = []
labels = []
with open('/content/drive/MyDrive/data_1.csv') as csvfile:
    data = list(csv.reader(csvfile))
    data = data[1:]
    for line in data:
        a, b, label = line[0], line[1], line[2]
        a_strings.append(a)
        b_strings.append(b)
        labels.append(label)

In [20]:
len(a_strings), len(b_strings), len(labels)

(3780, 3780, 3780)

In [21]:
# get the longest string length
max_len_a = max([len(s) for s in a_strings])
max_len_b = max([len(s) for s in b_strings])
print(max_len_a, max_len_b)

# max_str = max(strings, key=len)
# max_str

32 32


In [22]:
# get the unique labels
unique_labels = list(set(labels))
print(len(unique_labels))
unique_labels

60


['1f*k*',
 'h*4i*',
 '3*a*h',
 'l*ar*',
 'y*k*s',
 's*c3*',
 'e*z*z',
 '35*k*',
 'cp*8*',
 '7q*b*',
 'k*mg*',
 '9k*9*',
 '6*nw*',
 '9*1w*',
 '4*gg*',
 'h*b*5',
 'k0*h*',
 '0y*x*',
 'm*x*5',
 'wk*f*',
 'kz*d*',
 '4*5*c',
 '8*r*4',
 '2*p*s',
 '60*r*',
 'r*5*g',
 '5b*y*',
 'f*yu*',
 'p*lz*',
 'kx*d*',
 'w*i*d',
 '6*7j*',
 'c*h5*',
 'y*9*d',
 'a*re*',
 'q2*j*',
 '9*qw*',
 'z*to*',
 'r*ij*',
 'e*x*x',
 'e*u*q',
 'f*11*',
 'jq*2*',
 'g*t*2',
 'p*i3*',
 'pf*0*',
 'o*9*h',
 '4*gc*',
 '2*b*0',
 '1d*l*',
 'tc*m*',
 'zy*d*',
 'c*r*3',
 'q*ft*',
 '1i*s*',
 'u*3r*',
 'z*t*0',
 'q7*q*',
 'n*p*7',
 '1*y*s']

In [23]:
chars = {}
count = 1
for label in unique_labels:
  for char in label:
    if char not in chars:
      chars[char] = count
      count+=1
chars

{'1': 1,
 'f': 2,
 '*': 3,
 'k': 4,
 'h': 5,
 '4': 6,
 'i': 7,
 '3': 8,
 'a': 9,
 'l': 10,
 'r': 11,
 'y': 12,
 's': 13,
 'c': 14,
 'e': 15,
 'z': 16,
 '5': 17,
 'p': 18,
 '8': 19,
 '7': 20,
 'q': 21,
 'b': 22,
 'm': 23,
 'g': 24,
 '9': 25,
 '6': 26,
 'n': 27,
 'w': 28,
 '0': 29,
 'x': 30,
 'd': 31,
 '2': 32,
 'u': 33,
 'j': 34,
 't': 35,
 'o': 36}

In [24]:
def string_to_tensor(string, max_length=32):
    # Convert characters to numerical values
    char_to_int = chars
    tensor = [char_to_int[char] for char in string]

    # Pad the tensor to the max_length with 0
    tensor += [0] * (max_length - len(tensor))

    return tensor

def label_to_tensor(label, unique_labels):
    # Convert the label to a one-hot tensor
    label_tensor = [0] * len(unique_labels)
    label_tensor[unique_labels.index(label)] = 1

    return label_tensor

In [25]:
# Convert the strings to tensors
input_tensors_a = torch.from_numpy(np.array([string_to_tensor(s) for s in a_strings]))
input_tensors_b = torch.from_numpy(np.array([string_to_tensor(s) for s in b_strings]))
input_tensors_a.shape, input_tensors_b.shape

(torch.Size([3780, 32]), torch.Size([3780, 32]))

In [26]:
equals_vector = torch.reshape((einops.repeat(torch.tensor(0), " -> (i j)", i=p, j=105)), [-1,1])
add_vector = torch.reshape((einops.repeat(torch.tensor(37), " -> (i j)", i=p, j=105)), [-1,1])

In [27]:
add_vector.shape

torch.Size([3780, 1])

In [28]:
label_tensors = np.array([label_to_tensor(l, unique_labels) for l in labels])
label_tensors.shape

(3780, 60)

In [29]:
# print the first 5 input and label tensors
for i in range(5):
    print(input_tensors_a[i], input_tensors_b[i], label_tensors[i])

tensor([ 1, 31, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]) tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]) [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
tensor([ 1, 31, 31, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]) tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]) [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
tensor([ 1, 31, 31, 31, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]) tensor([10, 10, 10, 10, 10, 1

Input format:
|a|b|=|

In [30]:
dataset = torch.reshape(torch.cat((input_tensors_a, add_vector, input_tensors_b, equals_vector), dim=1), [3780, 66]).cuda()
print(dataset[:5])
print(dataset.shape)

tensor([[ 1, 31, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 37, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  0],
        [ 1, 31, 31, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 37, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  0],
        [ 1, 31, 31, 31, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 37, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  0],
        [ 1, 31, 31, 31, 31, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10,

In [31]:
# labels = (dataset[:, 0] + dataset[:, 1]) % p
# print(labels.shape)
# print(labels[:5])

labels = torch.tensor(label_tensors)
print(labels[:5])
print(labels.shape)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

Convert this to a train + test set - 30% in the training set

In [32]:
# torch.manual_seed(DATA_SEED)
# indices = torch.randperm(p*p)
# cutoff = int(p*p*frac_train)
# train_indices = indices[:cutoff]
# test_indices = indices[cutoff:]

# train_data = dataset[train_indices]
# train_labels = labels[train_indices]
# test_data = dataset[test_indices]
# test_labels = labels[test_indices]
# print(train_data[:5])
# print(train_labels[:5])
# print(train_data.shape)
# print(test_data[:5])
# print(test_labels[:5])
# print(test_data.shape)

# get the train and test data sampled randomly
torch.manual_seed(DATA_SEED)

# shuffle the data
indices = torch.randperm(len(dataset))
dataset = dataset[indices]
labels = labels[indices]

# split the data into train and test
train_cutoff = int(len(dataset) * frac_train)
train_data = dataset[:train_cutoff]
train_labels = labels[:train_cutoff]
test_data = dataset[train_cutoff:]
test_labels = labels[train_cutoff:]

In [33]:
# create attention masks
train_attention_mask = (train_data != 0).float()
test_attention_mask = (test_data != 0).float()

train_attention_mask[:5], test_attention_mask[:5]

(tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
   

In [34]:
print(train_data[:5])
print(train_labels[:5])

print(train_data.shape)
print(test_data.shape)


tensor([[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
         32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 37, 32, 32, 32,
         32, 32, 32, 32, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
         18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 13,  0],
        [ 1,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13,
         13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 37, 13, 13, 13,
         13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
         13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,  0],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 37, 10,  9, 11,
         11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
         11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  0],
        [14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
         14, 14,

In [35]:
from torch.utils.data import DataLoader, TensorDataset

In [36]:
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)


In [37]:
batch_size=64

# Create DataLoader for train and test data
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Define Model

In [38]:
cfg = HookedTransformerConfig(
    n_layers = 1,
    n_heads = 4,
    d_model = 128,
    d_head = 32,
    d_mlp = 512,
    act_fn = "relu",
    normalization_type=None,
    d_vocab=38, # Number of unique characters in vocab
    d_vocab_out=60, # Number of classes
    n_ctx=66,
    init_weights=True,
    # device="cuda",
    seed = 999,
)

In [39]:
model = HookedTransformer(cfg)

Disable the biases, as we don't need them for this task and it makes things easier to interpret.

In [40]:
for name, param in model.named_parameters():
    if "b_" in name:
        param.requires_grad = False


In [41]:
# # sample
# train_logits = model(train_data)
# print(train_logits.shape)
# print(train_logits[0])

## Define Optimizer + Loss

In [42]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=betas)

In [43]:
def loss_fn(logits, labels):
    if len(logits.shape)==3:
        logits = logits[:, -1]
    logits = logits.to(torch.float64)
    labels = labels.to(torch.float64)
    # do softmax
    logits = F.log_softmax(logits, dim=-1)
    # compute the cross entropy
    loss = nn.CrossEntropyLoss()
    return loss(logits, labels)

# train_logits = model.forward(train_data, attention_mask=train_attention_mask)
# train_loss = loss_fn(train_logits, train_labels)
# print(train_loss)
# test_logits = model(test_data)
# test_loss = loss_fn(test_logits, test_labels)
# print(test_loss)

In [44]:
print("Uniform loss:")
print(np.log(p))

Uniform loss:
3.58351893845611


## Actually Train

**Weird Decision:** Training the model with full batch training rather than stochastic gradient descent. We do this so to make training smoother and reduce the number of slingshots.

In [45]:
train_losses = []
test_losses = []
model_checkpoints = []
checkpoint_epochs = []
if TRAIN_MODEL:
    for epoch in tqdm.tqdm(range(num_epochs)):
        # Training loop
        model.train()
        batch_train_loss = []
        for batch_data, batch_labels in train_loader:
            optimizer.zero_grad()
            train_logits = model.forward(batch_data.cuda())
            train_loss = loss_fn(train_logits, batch_labels.cuda())
            train_loss.backward()
            optimizer.step()
            batch_train_loss.append(train_loss.item())
        train_losses.append(np.mean(batch_train_loss))

        # Testing loop
        model.eval()
        batch_test_loss = []
        with torch.inference_mode():
            for batch_data, batch_labels in test_loader:
                test_logits = model.forward(batch_data.cuda())
                test_loss = loss_fn(test_logits, batch_labels.cuda())
                batch_test_loss.append(test_loss.item())
            test_losses.append(np.mean(batch_test_loss))

        # Print and save checkpoints
        if ((epoch+1) % checkpoint_every) == 0:
            checkpoint_epochs.append(epoch)
            model_checkpoints.append(copy.deepcopy(model.state_dict()))
            print(f"Epoch {epoch} Train Loss {train_loss.item()} Test Loss {test_loss.item()}")

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 19 Train Loss 0.001997378638270335 Test Loss 0.02463672240055665
Epoch 39 Train Loss 0.00014255615529011696 Test Loss 0.023978382640503795
Epoch 59 Train Loss 0.000331630463114227 Test Loss 0.0012440661766838724
Epoch 79 Train Loss 0.0003654384853425715 Test Loss 0.07054306102804722
Epoch 99 Train Loss 0.00020563828933324094 Test Loss 0.00037665193279015434
Epoch 119 Train Loss 0.0001234120458366819 Test Loss 0.0003028386575875329
Epoch 139 Train Loss 2.0943850854099348e-05 Test Loss 4.9303902141194056e-05
Epoch 159 Train Loss 0.00019090381645464196 Test Loss 0.0015571753191511342
Epoch 179 Train Loss 8.692688366455833e-05 Test Loss 0.00020533350468931463
Epoch 199 Train Loss 4.165551160535649e-05 Test Loss 3.6301064970804456e-05


In [46]:
# prompt: do inference with test set and create classification report

# Do inference on the test set
model.eval()
test_logits = model(test_data)

if len(test_logits.shape) == 3:
    test_logits = test_logits[:, -1]

test_logits = torch.nn.functional.softmax(test_logits, dim=1)
test_logits = torch.argmax(test_logits, dim=1)

predicted_labels = torch.nn.functional.one_hot(test_logits, 60)
print(test_logits.shape)

from sklearn.metrics import classification_report, accuracy_score
print(classification_report(test_labels, predicted_labels.cpu().numpy()))
print(f"Accuracy: {accuracy_score(test_labels, predicted_labels.cpu().numpy()) * 100} %")


torch.Size([1134])
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       1.00      1.00      1.00        19
           2       0.92      0.92      0.92        13
           3       1.00      1.00      1.00        19
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00        14
           6       1.00      1.00      1.00        16
           7       1.00      0.95      0.98        21
           8       1.00      1.00      1.00        17
           9       1.00      1.00      1.00        12
          10       1.00      1.00      1.00        22
          11       1.00      1.00      1.00        21
          12       1.00      1.00      1.00        18
          13       0.95      1.00      0.97        19
          14       0.94      0.94      0.94        16
          15       1.00      1.00      1.00        23
          16       1.00      1.00      1.00        19
        

In [ ]:
torch.save(
    {
        "model":model.state_dict(),
        "config": model.cfg,
        "checkpoints": model_checkpoints,
        "checkpoint_epochs": checkpoint_epochs,
        "test_losses": test_losses,
        "train_losses": train_losses,
        # "train_indices": train_indices,
        # "test_indices": test_indices,
    },
    PTH_LOCATION)

In [ ]:
if not TRAIN_MODEL:
    cached_data = torch.load(PTH_LOCATION)
    model.load_state_dict(cached_data['model'])
    model_checkpoints = cached_data["checkpoints"]
    checkpoint_epochs = cached_data["checkpoint_epochs"]
    test_losses = cached_data['test_losses']
    train_losses = cached_data['train_losses']
    # train_indices = cached_data["train_indices"]
    # test_indices = cached_data["test_indices"]

## Show Model Training Statistics, Check that it groks!

In [47]:
%pip install git+https://github.com/neelnanda-io/neel-plotly.git
from neel_plotly.plot import line

  Cloning https://github.com/neelnanda-io/neel-plotly.git to /tmp/pip-req-build-8wetm8wn
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/neel-plotly.git /tmp/pip-req-build-8wetm8wn
  Resolved https://github.com/neelnanda-io/neel-plotly.git to commit 6dc24b26f8dec991908479d7445dae496b3430b7
  Preparing metadata (setup.py) ... done
  Created wheel for neel-plotly: filename=neel_plotly-0.0.0-py3-none-any.whl size=10186 sha256=ccb70e2b14f2e727b881b795070c908dcf9052fb6dc48f02dca50667bf047178
  Stored in directory: /tmp/pip-ephem-wheel-cache-vst5_cwf/wheels/32/cf/25/0103b4be02266c40faf008ffa9565a2ba07d1c63118fccc390
Successfully built neel-plotly


In [48]:
train_losses_2 = np.asarray(train_losses).astype('float32')
test_losses_2 = np.asarray(test_losses).astype('float32')
line([train_losses_2[::100], test_losses_2[::100]], x=np.arange(0, len(train_losses_2), 100), xaxis="Epoch", yaxis="Loss", log_y=True, title="Training Curve for Modular Addition", line_labels=['train', 'test'], toggle_x=True, toggle_y=True)

# Analysing the Model

In [ ]:
for i in dataset:
  print(i)
  break

## Standard Things to Try

In [52]:
original_logits, cache = model.run_with_cache(dataset)
print(original_logits.numel())

14968800


In [53]:
original_logits.shape

torch.Size([3780, 66, 60])

Get key weight matrices:

In [54]:
W_E = model.embed.W_E[:-1]
print("W_E", W_E.shape)
W_neur = W_E @ model.blocks[0].attn.W_V @ model.blocks[0].attn.W_O @ model.blocks[0].mlp.W_in
print("W_neur", W_neur.shape)
W_logit = model.blocks[0].mlp.W_out @ model.unembed.W_U
print("W_logit", W_logit.shape)

W_E torch.Size([37, 128])
W_neur torch.Size([4, 37, 512])
W_logit torch.Size([512, 60])


In [55]:
original_loss = loss_fn(original_logits, labels.to('cuda')).item()
print("Original Loss:", original_loss)

Original Loss: 0.019819642756646973


### Looking at Activations

Helper variable:

In [56]:
pattern_a = cache["pattern", 0, "attn"][:, :, -1, 0]
pattern_b = cache["pattern", 0, "attn"][:, :, -1, 1]
neuron_acts = cache["post", 0, "mlp"][:, -1, :]
neuron_pre_acts = cache["pre", 0, "mlp"][:, -1, :]

Get all shapes:

In [57]:
for param_name, param in cache.items():
    print(param_name, param.shape)

hook_embed torch.Size([3780, 66, 128])
hook_pos_embed torch.Size([3780, 66, 128])
blocks.0.hook_resid_pre torch.Size([3780, 66, 128])
blocks.0.attn.hook_q torch.Size([3780, 66, 4, 32])
blocks.0.attn.hook_k torch.Size([3780, 66, 4, 32])
blocks.0.attn.hook_v torch.Size([3780, 66, 4, 32])
blocks.0.attn.hook_attn_scores torch.Size([3780, 4, 66, 66])
blocks.0.attn.hook_pattern torch.Size([3780, 4, 66, 66])
blocks.0.attn.hook_z torch.Size([3780, 66, 4, 32])
blocks.0.hook_attn_out torch.Size([3780, 66, 128])
blocks.0.hook_resid_mid torch.Size([3780, 66, 128])
blocks.0.mlp.hook_pre torch.Size([3780, 66, 512])
blocks.0.mlp.hook_post torch.Size([3780, 66, 512])
blocks.0.hook_mlp_out torch.Size([3780, 66, 128])
blocks.0.hook_resid_post torch.Size([3780, 66, 128])


In [101]:
imshow(cache["pattern", 0].mean(dim=0)[:,-1,:], title="Average Attention Pattern per Head", xaxis="Source", yaxis="Head")

In [155]:
imshow(cache["pattern", 0][6][:, -1, :], title="String Specific Attention Pattern per Head", xaxis="Source", yaxis="Head",)

In [134]:
dataset[:66][:]

tensor([[32, 32, 32,  ..., 18, 13,  0],
        [ 1,  7,  7,  ..., 13, 13,  0],
        [10, 10, 10,  ..., 11, 11,  0],
        ...,
        [ 6,  6,  6,  ..., 14, 14,  0],
        [ 8,  8,  8,  ...,  9,  5,  0],
        [18, 18, 18,  ...,  8,  8,  0]], device='cuda:0')

In [109]:
cache["pattern", 0][6][0, -1, :].shape

torch.Size([66])

In [127]:
imshow(cache["pattern", 0][:66, 0, -1, :].reshape(66,66), title="Attention for Head 0", xaxis="characters", yaxis="string")

In [128]:
imshow(cache["pattern", 0][:66, 1, -1, :].reshape(66,66), title="Attention for Head 1", xaxis="characters", yaxis="string")

In [129]:
imshow(cache["pattern", 0][:66, 2, -1, :].reshape(66,66), title="Attention for Head 2", xaxis="characters", yaxis="string")

In [130]:
imshow(cache["pattern", 0][:66, 3, -1, :].reshape(66,66), title="Attention for Head 3", xaxis="characters", yaxis="string")

In [135]:
# imshow(
#     einops.rearrange(cache["pattern", 0][:66, :, -1, :], "(a b) head -> head a b", a= 66, b=66),
#     title="Attention for Head 0 from a -> =", xaxis="b", yaxis="a", facet_col=0)

Plotting neuron activations

In [69]:
cache["post", 0, "mlp"].shape

torch.Size([3780, 66, 512])

In [81]:
cache["pattern",0].shape

torch.Size([3780, 4, 66, 66])

In [85]:
cache["pattern",0][:, 0, :, :].shape

torch.Size([3780, 66, 66])

In [153]:
neuron_acts[:32][:32].shape

torch.Size([32, 512])

In [157]:
imshow(neuron_acts[:32,:32].reshape(32,32), xaxis = 'neuron', yaxis='string')

### Singular Value Decomposition

In [158]:
W_E.shape

torch.Size([37, 128])

In [159]:
U, S, Vh = torch.svd(W_E)
line(S, title="Singular Values")
imshow(U, title="Principal Components on the Input")

In [160]:
# Control - random Gaussian matrix
U, S, Vh = torch.svd(torch.randn_like(W_E))
line(S, title="Singular Values Random")
imshow(U, title="Principal Components Random")

## Explaining Algorithm

### Analyse the Embedding - It's a Lookup Table!

In [161]:
U, S, Vh = torch.svd(W_E)
line(U[:, :8].T, title="Principal Components of the embedding", xaxis="Input Vocabulary")

In [162]:
fourier_basis = []
fourier_basis_names = []
fourier_basis.append(torch.ones(p))
fourier_basis_names.append("Constant")
for freq in range(1, p//2+1):
    fourier_basis.append(torch.sin(torch.arange(p)*2 * torch.pi * freq / p))
    fourier_basis_names.append(f"Sin {freq}")
    fourier_basis.append(torch.cos(torch.arange(p)*2 * torch.pi * freq / p))
    fourier_basis_names.append(f"Cos {freq}")
fourier_basis = torch.stack(fourier_basis, dim=0).cuda()
fourier_basis = fourier_basis/fourier_basis.norm(dim=-1, keepdim=True)
imshow(fourier_basis, xaxis="Input", yaxis="Component", y=fourier_basis_names)

In [163]:
line(fourier_basis[:8], xaxis="Input", line_labels=fourier_basis_names[:8], title="First 8 Fourier Components")
line(fourier_basis[25:29], xaxis="Input", line_labels=fourier_basis_names[25:29], title="Middle Fourier Components")

In [ ]:
imshow(fourier_basis @ fourier_basis.T, title="All Fourier Vectors are Orthogonal")

### Analyse the Embedding

In [ ]:
imshow(fourier_basis @ W_E, yaxis="Fourier Component", xaxis="Residual Stream", y=fourier_basis_names, title="Embedding in Fourier Basis")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (51x51 and 63x128)

In [ ]:
line((fourier_basis @ W_E).norm(dim=-1), xaxis="Fourier Component", x=fourier_basis_names, title="Norms of Embedding in Fourier Basis")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (51x51 and 63x128)

In [ ]:
key_freqs = [17, 25, 32, 47]
key_freq_indices = [33, 34, 49, 50, 63, 64, 93, 94]
fourier_embed = fourier_basis @ W_E
key_fourier_embed = fourier_embed[key_freq_indices]
print("key_fourier_embed", key_fourier_embed.shape)
imshow(key_fourier_embed @ key_fourier_embed.T, title="Dot Product of embedding of key Fourier Terms")

### Key Frequencies

In [ ]:
line(fourier_basis[[34, 50, 64, 94]], title="Cos of key freqs", line_labels=[34, 50, 64, 94])

In [ ]:
line(fourier_basis[[34, 50, 64, 94]].mean(0), title="Constructive Interference")

## Analyse Neurons

In [ ]:
imshow(
    einops.rearrange(neuron_acts[:, :5], "(a b) neuron -> neuron a b", a=p, b=p),
    title="First 5 neuron acts", xaxis="b", yaxis="a", facet_col=0)

In [ ]:
imshow(
    einops.rearrange(neuron_acts[:, 0], "(a b) -> a b", a=p, b=p),
    title="First neuron act", xaxis="b", yaxis="a",)

In [ ]:
imshow(fourier_basis[94][None, :] * fourier_basis[94][:, None], title="Cos 47a * cos 47b")

In [ ]:
imshow(fourier_basis[94][None, :] * fourier_basis[0][:, None], title="Cos 47a * const")

In [ ]:
imshow(fourier_basis @ neuron_acts[:, 0].reshape(p, p) @ fourier_basis.T, title="2D Fourier Transformer of neuron 0", xaxis="b", yaxis="a", x=fourier_basis_names, y=fourier_basis_names)

In [ ]:
imshow(fourier_basis @ neuron_acts[:, 5].reshape(p, p) @ fourier_basis.T, title="2D Fourier Transformer of neuron 5", xaxis="b", yaxis="a", x=fourier_basis_names, y=fourier_basis_names)

In [ ]:
imshow(fourier_basis @ torch.randn_like(neuron_acts[:, 0]).reshape(p, p) @ fourier_basis.T, title="2D Fourier Transformer of RANDOM", xaxis="b", yaxis="a", x=fourier_basis_names, y=fourier_basis_names)

### Neuron Clusters

In [ ]:
fourier_neuron_acts = fourier_basis @ einops.rearrange(neuron_acts, "(a b) neuron -> neuron a b", a=p, b=p) @ fourier_basis.T
# Center these by removing the mean - doesn't matter!
fourier_neuron_acts[:, 0, 0] = 0.
print("fourier_neuron_acts", fourier_neuron_acts.shape)

In [ ]:
neuron_freq_norm = torch.zeros(p//2, model.cfg.d_mlp).cuda()
for freq in range(0, p//2):
    for x in [0, 2*(freq+1) - 1, 2*(freq+1)]:
        for y in [0, 2*(freq+1) - 1, 2*(freq+1)]:
            neuron_freq_norm[freq] += fourier_neuron_acts[:, x, y]**2
neuron_freq_norm = neuron_freq_norm / fourier_neuron_acts.pow(2).sum(dim=[-1, -2])[None, :]
imshow(neuron_freq_norm, xaxis="Neuron", yaxis="Freq", y=torch.arange(1, p//2+1), title="Neuron Frac Explained by Freq")

In [ ]:
line(neuron_freq_norm.max(dim=0).values.sort().values, xaxis="Neuron", title="Max Neuron Frac Explained over Freqs")

## Read Off the Neuron-Logit Weights to Interpret

In [ ]:
W_logit = model.blocks[0].mlp.W_out @ model.unembed.W_U
print("W_logit", W_logit.shape)

In [ ]:
line((W_logit @ fourier_basis.T).norm(dim=0), x=fourier_basis_names, title="W_logit in the Fourier Basis")

In [ ]:
neurons_17 = neuron_freq_norm[17-1]>0.85
neurons_17.shape

In [ ]:
neurons_17.sum()

In [ ]:
line((W_logit[neurons_17] @ fourier_basis.T).norm(dim=0), x=fourier_basis_names, title="W_logit for freq 17 neurons in the Fourier Basis")

Study sin 17

In [ ]:
freq = 17
W_logit_fourier = W_logit @ fourier_basis
neurons_sin_17 = W_logit_fourier[:, 2*freq-1]
line(neurons_sin_17)

In [ ]:
neuron_acts.shape

In [ ]:
inputs_sin_17c = neuron_acts @ neurons_sin_17
imshow(fourier_basis @ inputs_sin_17c.reshape(p, p) @ fourier_basis.T, title="Fourier Heatmap over inputs for sin17c", x=fourier_basis_names, y=fourier_basis_names)

# Black Box Methods + Progress Measures

## Setup Code

Code to plot embedding freqs

In [ ]:
def embed_to_cos_sin(fourier_embed):
    if len(fourier_embed.shape) == 1:
        return torch.stack([fourier_embed[1::2], fourier_embed[2::2]])
    else:
        return torch.stack([fourier_embed[:, 1::2], fourier_embed[:, 2::2]], dim=1)

from neel_plotly.plot import melt

def plot_embed_bars(
    fourier_embed,
    title="Norm of embedding of each Fourier Component",
    return_fig=False,
    **kwargs
):
    cos_sin_embed = embed_to_cos_sin(fourier_embed)
    df = melt(cos_sin_embed)
    # display(df)
    group_labels = {0: "sin", 1: "cos"}
    df["Trig"] = df["0"].map(lambda x: group_labels[x])
    fig = px.bar(
        df,
        barmode="group",
        color="Trig",
        x="1",
        y="value",
        labels={"1": "$w_k$", "value": "Norm"},
        title=title,
        **kwargs
    )
    fig.update_layout(dict(legend_title=""))

    if return_fig:
        return fig
    else:
        fig.show()

Code to test a tensor of edited logits

In [ ]:
def test_logits(logits, bias_correction=False, original_logits=None, mode="all"):
    # Calculates cross entropy loss of logits representing a batch of all p^2
    # possible inputs
    # Batch dimension is assumed to be first
    if logits.shape[1] == p * p:
        logits = logits.T
    if logits.shape == torch.Size([p * p, p + 1]):
        logits = logits[:, :-1]
    logits = logits.reshape(p * p, p)
    if bias_correction:
        # Applies bias correction - we correct for any missing bias terms,
        # independent of the input, by centering the new logits along the batch
        # dimension, and then adding the average original logits across all inputs
        logits = (
            einops.reduce(original_logits - logits, "batch ... -> ...", "mean") + logits
        )
    if mode == "train":
        return loss_fn(logits[train_indices], labels[train_indices])
    elif mode == "test":
        return loss_fn(logits[test_indices], labels[test_indices])
    elif mode == "all":
        return loss_fn(logits, labels)

Code to run a metric over every checkpoint

In [ ]:
metric_cache = {}

In [ ]:
def get_metrics(model, metric_cache, metric_fn, name, reset=False):
    if reset or (name not in metric_cache) or (len(metric_cache[name]) == 0):
        metric_cache[name] = []
        for c, sd in enumerate(tqdm.tqdm((model_checkpoints))):
            model.reset_hooks()
            model.load_state_dict(sd)
            out = metric_fn(model)
            if type(out) == torch.Tensor:
                out = utils.to_numpy(out)
            metric_cache[name].append(out)
        model.load_state_dict(model_checkpoints[-1])
        try:
            metric_cache[name] = torch.tensor(metric_cache[name])
        except:
            metric_cache[name] = torch.tensor(np.array(metric_cache[name]))



## Defining Progress Measures

### Loss Curves

In [ ]:
memorization_end_epoch = 1500
circuit_formation_end_epoch = 13300
cleanup_end_epoch = 16600

In [ ]:
def add_lines(figure):
    figure.add_vline(memorization_end_epoch, line_dash="dash", opacity=0.7)
    figure.add_vline(circuit_formation_end_epoch, line_dash="dash", opacity=0.7)
    figure.add_vline(cleanup_end_epoch, line_dash="dash", opacity=0.7)
    return figure

In [ ]:
fig = line([train_losses[::100], test_losses[::100]], x=np.arange(0, len(train_losses), 100), xaxis="Epoch", yaxis="Loss", log_y=True, title="Training Curve for Modular Addition", line_labels=['train', 'test'], toggle_x=True, toggle_y=True, return_fig=True)
add_lines(fig)

### Logit Periodicity

In [ ]:
all_logits = original_logits[:, -1, :]
print(all_logits.shape)
all_logits = einops.rearrange(all_logits, "(a b) c -> a b c", a=p, b=p)
print(all_logits.shape)

In [ ]:
coses = {}
for freq in key_freqs:
    print("Freq:", freq)
    a = torch.arange(p)[:, None, None]
    b = torch.arange(p)[None, :, None]
    c = torch.arange(p)[None, None, :]
    cube_predicted_logits = torch.cos(freq * 2 * torch.pi / p * (a + b - c)).cuda()
    cube_predicted_logits /= cube_predicted_logits.norm()
    coses[freq] = cube_predicted_logits

In [ ]:
approximated_logits = torch.zeros_like(all_logits)
for freq in key_freqs:
    print("Freq:", freq)
    coeff = (all_logits * coses[freq]).sum()
    print("Coeff:", coeff)
    cosine_sim = coeff / all_logits.norm()
    print("Cosine Sim:", cosine_sim)
    approximated_logits += coeff * coses[freq]
residual = all_logits - approximated_logits
print("Residual size:", residual.norm())
print("Residual fraction of norm:", residual.norm()/all_logits.norm())

In [ ]:
random_logit_cube = torch.randn_like(all_logits)
print((all_logits * random_logit_cube).sum()/random_logit_cube.norm()/all_logits.norm())

In [ ]:
test_logits(all_logits)

In [ ]:
test_logits(approximated_logits)

#### Look During Training

In [ ]:
cos_cube = []
for freq in range(1, p//2 + 1):
    a = torch.arange(p)[:, None, None]
    b = torch.arange(p)[None, :, None]
    c = torch.arange(p)[None, None, :]
    cube_predicted_logits = torch.cos(freq * 2 * torch.pi / p * (a + b - c)).cuda()
    cube_predicted_logits /= cube_predicted_logits.norm()
    cos_cube.append(cube_predicted_logits)
cos_cube = torch.stack(cos_cube, dim=0)
print(cos_cube.shape)

In [ ]:
def get_cos_coeffs(model):
    logits = model(dataset)[:, -1]
    logits = einops.rearrange(logits, "(a b) c -> a b c", a=p, b=p)
    vals = (cos_cube * logits[None, :, :, :]).sum([-3, -2, -1])
    return vals


get_metrics(model, metric_cache, get_cos_coeffs, "cos_coeffs")
print(metric_cache["cos_coeffs"].shape)

In [ ]:
fig = line(metric_cache["cos_coeffs"].T, line_labels=[f"Freq {i}" for i in range(1, p//2+1)], title="Coefficients with Predicted Logits", xaxis="Epoch", x=checkpoint_epochs, yaxis="Coefficient", return_fig=True)
add_lines(fig)

In [ ]:
def get_cos_sim(model):
    logits = model(dataset)[:, -1]
    logits = einops.rearrange(logits, "(a b) c -> a b c", a=p, b=p)
    vals = (cos_cube * logits[None, :, :, :]).sum([-3, -2, -1])
    return vals / logits.norm()

get_metrics(model, metric_cache, get_cos_sim, "cos_sim") # You may need a big GPU. If you don't have one and can't work around this, raise an issue for help!
print(metric_cache["cos_sim"].shape)

fig = line(metric_cache["cos_sim"].T, line_labels=[f"Freq {i}" for i in range(1, p//2+1)], title="Cosine Sim with Predicted Logits", xaxis="Epoch", x=checkpoint_epochs, yaxis="Cosine Sim", return_fig=True)
add_lines(fig)

In [ ]:
def get_residual_cos_sim(model):
    logits = model(dataset)[:, -1]
    logits = einops.rearrange(logits, "(a b) c -> a b c", a=p, b=p)
    vals = (cos_cube * logits[None, :, :, :]).sum([-3, -2, -1])
    residual = logits - (vals[:, None, None, None] * cos_cube).sum(dim=0)
    return residual.norm() / logits.norm()

get_metrics(model, metric_cache, get_residual_cos_sim, "residual_cos_sim")
print(metric_cache["residual_cos_sim"].shape)

fig = line([metric_cache["cos_sim"][:, i] for i in range(p//2)]+[metric_cache["residual_cos_sim"]], line_labels=[f"Freq {i}" for i in range(1, p//2+1)]+["residual"], title="Cosine Sim with Predicted Logits + Residual", xaxis="Epoch", x=checkpoint_epochs, yaxis="Cosine Sim", return_fig=True)
add_lines(fig)

## Restricted Loss

In [ ]:
neuron_acts.shape

In [ ]:
neuron_acts_square = einops.rearrange(neuron_acts, "(a b) neur -> a b neur", a=p, b=p).clone()
# Center it
neuron_acts_square -= einops.reduce(neuron_acts_square, "a b neur -> 1 1 neur", "mean")
neuron_acts_square_fourier = einsum("a b neur, fa a, fb b -> fa fb neur", neuron_acts_square, fourier_basis, fourier_basis)
imshow(neuron_acts_square_fourier.norm(dim=-1), xaxis="Fourier Component b", yaxis="Fourier Component a", title="Norms of neuron activations by Fourier Component", x=fourier_basis_names, y=fourier_basis_names)

In [ ]:
original_logits, cache = model.run_with_cache(dataset)
print(original_logits.numel())
neuron_acts = cache["post", 0, "mlp"][:, -1, :]

In [ ]:
approx_neuron_acts = torch.zeros_like(neuron_acts)
approx_neuron_acts += neuron_acts.mean(dim=0)
a = torch.arange(p)[:, None]
b = torch.arange(p)[None, :]
for freq in key_freqs:
    cos_apb_vec = torch.cos(freq * 2 * torch.pi / p * (a + b)).cuda()
    cos_apb_vec /= cos_apb_vec.norm()
    cos_apb_vec = einops.rearrange(cos_apb_vec, "a b -> (a b) 1")
    approx_neuron_acts += (neuron_acts * cos_apb_vec).sum(dim=0) * cos_apb_vec
    sin_apb_vec = torch.sin(freq * 2 * torch.pi / p * (a + b)).cuda()
    sin_apb_vec /= sin_apb_vec.norm()
    sin_apb_vec = einops.rearrange(sin_apb_vec, "a b -> (a b) 1")
    approx_neuron_acts += (neuron_acts * sin_apb_vec).sum(dim=0) * sin_apb_vec
restricted_logits = approx_neuron_acts @ W_logit
print(loss_fn(restricted_logits[test_indices], test_labels))

In [ ]:
print(loss_fn(all_logits, labels)) # This bugged on models not fully trained

### Look During Training

In [ ]:
def get_restricted_loss(model):
    logits, cache = model.run_with_cache(dataset)
    logits = logits[:, -1, :]
    neuron_acts = cache["post", 0, "mlp"][:, -1, :]
    approx_neuron_acts = torch.zeros_like(neuron_acts)
    approx_neuron_acts += neuron_acts.mean(dim=0)
    a = torch.arange(p)[:, None]
    b = torch.arange(p)[None, :]
    for freq in key_freqs:
        cos_apb_vec = torch.cos(freq * 2 * torch.pi / p * (a + b)).cuda()
        cos_apb_vec /= cos_apb_vec.norm()
        cos_apb_vec = einops.rearrange(cos_apb_vec, "a b -> (a b) 1")
        approx_neuron_acts += (neuron_acts * cos_apb_vec).sum(dim=0) * cos_apb_vec
        sin_apb_vec = torch.sin(freq * 2 * torch.pi / p * (a + b)).cuda()
        sin_apb_vec /= sin_apb_vec.norm()
        sin_apb_vec = einops.rearrange(sin_apb_vec, "a b -> (a b) 1")
        approx_neuron_acts += (neuron_acts * sin_apb_vec).sum(dim=0) * sin_apb_vec
    restricted_logits = approx_neuron_acts @ model.blocks[0].mlp.W_out @ model.unembed.W_U
    # Add bias term
    restricted_logits += logits.mean(dim=0, keepdim=True) - restricted_logits.mean(dim=0, keepdim=True)
    return loss_fn(restricted_logits[test_indices], test_labels)
get_restricted_loss(model)

In [ ]:
get_metrics(model, metric_cache, get_restricted_loss, "restricted_loss", reset=True)
print(metric_cache["restricted_loss"].shape)

In [ ]:
fig = line([train_losses[::100], test_losses[::100], metric_cache["restricted_loss"]], x=np.arange(0, len(train_losses), 100), xaxis="Epoch", yaxis="Loss", log_y=True, title="Restricted Loss Curve", line_labels=['train', 'test', "restricted_loss"], toggle_x=True, toggle_y=True, return_fig=True)
add_lines(fig)

In [ ]:
fig = line([torch.tensor(test_losses[::100])/metric_cache["restricted_loss"]], x=np.arange(0, len(train_losses), 100), xaxis="Epoch", yaxis="Loss", log_y=True, title="Restricted Loss to Test Loss Ratio", toggle_x=True, toggle_y=True, return_fig=True)
# WARNING: bugged when cancelling training half way thr ough
add_lines(fig)

## Excluded Loss

In [ ]:
approx_neuron_acts = torch.zeros_like(neuron_acts)
# approx_neuron_acts += neuron_acts.mean(dim=0)
a = torch.arange(p)[:, None]
b = torch.arange(p)[None, :]
for freq in key_freqs:
    cos_apb_vec = torch.cos(freq * 2 * torch.pi / p * (a + b)).cuda()
    cos_apb_vec /= cos_apb_vec.norm()
    cos_apb_vec = einops.rearrange(cos_apb_vec, "a b -> (a b) 1")
    approx_neuron_acts += (neuron_acts * cos_apb_vec).sum(dim=0) * cos_apb_vec
    sin_apb_vec = torch.sin(freq * 2 * torch.pi / p * (a + b)).cuda()
    sin_apb_vec /= sin_apb_vec.norm()
    sin_apb_vec = einops.rearrange(sin_apb_vec, "a b -> (a b) 1")
    approx_neuron_acts += (neuron_acts * sin_apb_vec).sum(dim=0) * sin_apb_vec
excluded_neuron_acts = neuron_acts - approx_neuron_acts
excluded_logits = excluded_neuron_acts @ W_logit
print(loss_fn(excluded_logits[train_indices], train_labels))

In [ ]:
def get_excluded_loss(model):
    logits, cache = model.run_with_cache(dataset)
    logits = logits[:, -1, :]
    neuron_acts = cache["post", 0, "mlp"][:, -1, :]
    approx_neuron_acts = torch.zeros_like(neuron_acts)
    # approx_neuron_acts += neuron_acts.mean(dim=0)
    a = torch.arange(p)[:, None]
    b = torch.arange(p)[None, :]
    for freq in key_freqs:
        cos_apb_vec = torch.cos(freq * 2 * torch.pi / p * (a + b)).cuda()
        cos_apb_vec /= cos_apb_vec.norm()
        cos_apb_vec = einops.rearrange(cos_apb_vec, "a b -> (a b) 1")
        approx_neuron_acts += (neuron_acts * cos_apb_vec).sum(dim=0) * cos_apb_vec
        sin_apb_vec = torch.sin(freq * 2 * torch.pi / p * (a + b)).cuda()
        sin_apb_vec /= sin_apb_vec.norm()
        sin_apb_vec = einops.rearrange(sin_apb_vec, "a b -> (a b) 1")
        approx_neuron_acts += (neuron_acts * sin_apb_vec).sum(dim=0) * sin_apb_vec
    excluded_neuron_acts = neuron_acts - approx_neuron_acts
    residual_stream_final = excluded_neuron_acts @ model.blocks[0].mlp.W_out + cache["resid_mid", 0][:, -1, :]
    excluded_logits = residual_stream_final @ model.unembed.W_U
    return loss_fn(excluded_logits[train_indices], train_labels)
get_excluded_loss(model)

In [ ]:
get_metrics(model, metric_cache, get_excluded_loss, "excluded_loss", reset=True)
print(metric_cache["excluded_loss"].shape)

In [ ]:
fig = line([train_losses[::100], test_losses[::100], metric_cache["excluded_loss"], metric_cache["restricted_loss"]], x=np.arange(0, len(train_losses), 100), xaxis="Epoch", yaxis="Loss", log_y=True, title="Excluded and Restricted Loss Curve", line_labels=['train', 'test', "excluded_loss", "restricted_loss"], toggle_x=True, toggle_y=True, return_fig=True)

add_lines(fig)